In [2]:
"""
This is a story about asynchronous!
临时学习能力考察:)
"""

from utils import Section
from collections import Generator

with Section("""
    Do you know Python generator?
"""):

    def generator(x, inc):
        yield x
        yield x + inc
        yield x + inc + inc
        return "Done"

    # here're some behaviours of a Python generator
    
    gen = generator(1, inc=2)

    lst = []
    try:
        while True:
            lst.append(next(gen))
    except StopIteration as e:
        ret = e.value

    assert lst == [1, 3, 5]
    assert ret == "Done"

    
    

with Section("""

    Now You have got what's a generator in Python, so could you solve
    the following problems?
    
    
    - If I have 3 generators and I want to drive them
      forward synchronously.
    
    e.g
        gen1: 1, 2, 3
        gen2: "1", "11", "111"
        sync(gen1, gen2): 1, "1", 2, "11", 3, "111"

"""):

    def sync(*gens) -> Generator:
        """
        ** 实现这个函数: 以下实现为参考
        """
        remaining_event_ids = set(range(len(gens)))

        while remaining_event_ids:
            for gen_id in tuple(remaining_event_ids):
                try:
                    yield next(gens[gen_id])

                except StopIteration:
                    remaining_event_ids.remove(gen_id)

    gen1 = generator(1, inc=1)
    gen2 = generator("1", inc="1")

    gen3 = sync(gen1, gen2)


    
with Section("""Test your implementation!"""):
    assert next(gen3) == 1
    assert next(gen3) == '1'
    assert next(gen3) == 2
    assert next(gen3) == '11'


    
    
    
with Section("""
    Now you have got how to synchronize the generators, 
    let's learn what's `async` exactly!
            """):
    import time
    import random

    class Query:
        def __init__(self):

            self.time_cost = random.randint(0, 5)
            self.start = time.time()

        @property  # 受试者需了解基本的property用法才能看懂。property是python控制数据访问权限的重要方式
        def is_finished(self):
            return time.time() - self.start > self.time_cost

        @property
        def result(self):
            return "data fetched"

        def reset(self):
            self.start = time.time()

    # Assume that I have made multiple data queries with function `query`,
    q1 = Query()
    q2 = Query()
    q3 = Query()

    # Each query differs in time cost.
    # How could I be able to assure I can get all the 3 query results as fast as possible.

    # Requirement:
    # Implement the function `feed_back` to monitor the 3 query to return the 3 results within least time period.


    def feed_back():
        """
        ** 实现这个函数: 以下实现为参考
        """
        result = [None, None, None]
        qs = [q1, q2, q3]
        remaining_ids = {0, 1, 2}
        while remaining_ids:
            for remaining_id in tuple(remaining_ids):
                q = qs[remaining_id]
                if q.is_finished:
                    result[remaining_id] = q.result
                    remaining_ids.remove(remaining_id)

        return result

    assert feed_back() == ['data fetched'] * 3

    # Okay, you've solved above problem, that's cool!
    #
    # However, what if you have to pay attention to other tasks when you're monitoring the 3 queries.
    class OtherTask(Query):
        def __init__(self):
            super().__init__()

    q1.reset()
    q2.reset()
    q3.reset()
    other_task = OtherTask()

    # A real world problem is presented here:
    #
    #       Sometimes, each query might cost a very long time, for example, 10 minutes,
    #       while the other tasks would take you a while as well.
    #
    #       Assume that there is no overhead to check if a query is finished,
    #       obviously it's not difficult to get that if you got blocked at checking queries, you might
    #       waste 10 minutes producing nothing.
    #
    # That's the reason why we need `async`.

    # 受试者需读懂以下连续4个函数才能会做下一题。
    # 四个函数复习了这一题中提到的所有知识。

    def async_feed_back():
        result = [None, None, None]
        qs = [q1, q2, q3]
        remaining_ids = {0, 1, 2}
        while remaining_ids:
            for remaining_id in tuple(remaining_ids):
                q = qs[remaining_id]
                if q.is_finished:
                    result[remaining_id] = q.result
                    remaining_ids.remove(remaining_id)
                else:
                    yield

        return result

    def async_other_tasks():
        """
        process other task for a short time, and then just leave right now."
        """
        while not other_task.is_finished:
            yield

        return other_task.result

    def sync_return(*gens):
        
        remaining_event_ids = set(range(len(gens)))
        
        results = [None] * len(gens)
        
        while remaining_event_ids:
        
            for gen_id in tuple(remaining_event_ids):
                
                try:
                
                    yield next(gens[gen_id])

                except StopIteration as e:
                    
                    results[gen_id] = e.value
                    
                    remaining_event_ids.remove(gen_id)
        
        return results

    def get_async_result(gen):
        
        try:
            while True:
                next(gen)
        
        except StopIteration as e:
            return e.value

    gathered_tasks = sync_return(async_feed_back(), async_other_tasks())
    
    query_results, other_task_result = get_async_result(gathered_tasks)
    
    print(query_results, other_task_result)

# Now let's do something that solves some meaningful tasks in the real world!
# 和现实联系。
# 异步爬取推特动态。


with Section(""""Get people's recent tweets from twitter!"""):
    from utils import find_recent_tweets
    
    # FYI:
    #
    # `find_recent_tweets` is an API for you to asynchronously load an user's recent tweets from
    # `https://twitter.com/<username>`.

    # signature =>    
    #     find_recent_tweets: (username: string) -> generator, 
    #                        where this generator function returns the user's recent tweets.
    

    def my_async_query_from_twitter(users_to_ask=('bing', 'microsoft', 'azure')):
        """
        :return: a list of string represents the recent tweets from corresponding users. 
        
        ** 实现这个函数。以下实现为参考。
        """
    
        remaining_event_ids, event_loop = zip(
            *enumerate(map(find_recent_tweets, users_to_ask)))

        remaining_event_ids = set(remaining_event_ids)

        results = [None] * len(event_loop)

        while remaining_event_ids:
            
            for idx in tuple(remaining_event_ids):
                
                event = event_loop[idx]

                try:
                    next(event)
                
                except StopIteration as e:
                    
                    result = results[idx] = e.value
                    print(f'got tweets from {users_to_ask[idx]}: {result}')

                    remaining_event_ids.remove(idx)
                    
        return results
    
    for each in my_async_query_from_twitter():
        print(each)


['data fetched', 'data fetched', 'data fetched'] data fetched


c:\users\twshe\software\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\twshe\software\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


got tweets from bing: 
Road trips are better when you have a co-pilot, and co-pilots are better when they have the Bing app. Who’s your co-pilot this summer?pic.twitter.com/zWJQeaIGKT

got tweets from microsoft: 
This is no ordinary cat video. This one explains #quantumcomputing in less than 4 minutes. Watch meow:pic.twitter.com/si7U1rbH5H

got tweets from azure: 
Focus on the core development of your #app by deploying reliable distributed systems. Download our eBook to find out how: http://msft.social/geruW6 pic.twitter.com/WB0hjQ6Eh0


Road trips are better when you have a co-pilot, and co-pilots are better when they have the Bing app. Who’s your co-pilot this summer?pic.twitter.com/zWJQeaIGKT


This is no ordinary cat video. This one explains #quantumcomputing in less than 4 minutes. Watch meow:pic.twitter.com/si7U1rbH5H


Focus on the core development of your #app by deploying reliable distributed systems. Download our eBook to find out how: http://msft.social/geruW6 pic.twitter.co